In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
EXECUTABLE_NAME_SCENARIO = 'cos_dist'
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"

# random number generator
rng = np.random.default_rng()

In [2]:
def run_sfe(x, y, y_0, y_1):
    # write the original 2 vectors to a file (second vector used only for verification)
    with open(f"{EXECUTABLE_PATH}/{INPUT_FILE_NAME}", 'w') as f:
        for x_i, y_i in zip(x, y):
            f.write(f"{x_i} {y_i}\n")
            
    # write the shares into separate files
    with open(f"{EXECUTABLE_PATH}/share0.txt", 'w') as f:
        for i in y_0:
            f.write(f"{i}\n")
    with open(f"{EXECUTABLE_PATH}/share1.txt", 'w') as f:
        for i in y_1:
            f.write(f"{i}\n")
            
    # execute the ABY cos sim computation
    output = subprocess.run(CMD_SCENARIO, shell=True, capture_output=True, text=True, cwd=EXECUTABLE_PATH)
    assert (output.returncode == 0) # make sure the process executed successfully
    
    return output

def get_embedding(imagepath):
    return DeepFace.represent(img_path = imagepath, model_name="SFace", enforce_detection=True)[0]["embedding"]
        
def get_two_random_embeddings(same_person):
    """Get two random embeddings of either the same person or two different people out of all the images available"""
    people = os.listdir('lfw') # list of all people that have images
    people_with_multiple_images = [p for p in people if len(os.listdir(f"lfw/{p}")) > 1] # list of people with more than one image in folder
    embedding1, embedding2 = None, None # face embeddings
    while embedding1 is None or embedding2 is None: # try until the chosen images have detectable faces
        try:
            if same_person:
                # same person should have more than one image (we might still end up choosing the same image of that person with prob 1/n, but that's ok)
                person1 = random.choice(people_with_multiple_images)
                person2 = person1
            else:
                # two persons chosen should be different
                person1 = random.choice(people)
                person2 = random.choice([p for p in people if p != person1])
            # get two random images
            img1 = f"lfw/{person1}/{random.choice(os.listdir(f'lfw/{person1}'))}"
            img2 = f"lfw/{person2}/{random.choice(os.listdir(f'lfw/{person2}'))}"
            # try to extract embeddings from both images
            embedding1 = get_embedding(img1)
            embedding2 = get_embedding(img2)
        except Exception as e:
            # failed to detect faces in images, try again
            # print(e)
            pass
    return np.array(embedding1), np.array(embedding2)

In [3]:
# 1st try with real data

# Get two embeddings of images that we will be comparing.
a, b = get_two_random_embeddings(same_person=False)

# This is how they look like raw
a

array([-1.05887663,  1.1577996 ,  0.6159687 ,  0.92861074,  0.54197681,
        0.1466969 , -0.98524421,  0.57258457, -0.44633129,  0.98824561,
        0.98547626,  0.61538422,  1.88092852, -0.75701547, -0.3482213 ,
       -0.73323172, -0.31149465,  0.10216737,  0.19858135,  0.78104699,
       -0.42418146,  0.74085951, -0.4718197 , -0.35617095, -0.04339859,
        1.2745738 ,  0.19090196,  0.57039332, -0.29325372,  1.48540282,
       -0.71376383,  1.09925592,  1.79980183,  0.40419835, -0.083152  ,
        0.78684235, -0.10030162,  1.45961678, -0.71542764,  0.4785786 ,
       -0.38525358, -0.11947311,  1.152475  ,  0.50615567, -0.85845679,
       -0.46455294, -0.2670826 ,  0.45135278,  0.74665898,  0.57165748,
       -0.49452114, -0.58083856, -0.16251093,  0.16182835,  0.66316724,
        0.72607446, -0.01495208,  0.23632735,  0.4999547 ,  0.38669533,
        1.13880157,  0.30691868,  0.71149141, -1.86636066, -1.14623201,
       -0.19288938,  0.21882766,  0.36012024,  0.49171108, -0.19

In [4]:
# Make the shares
# x is the captured face, y is the face in the database
# First, scale the values up by 10 000, then get rid of the decimal part then cast to int

x = (a * 10000).round().astype(int)
y = (b * 10000).round().astype(int)

# so far so good
x

array([-10589,  11578,   6160,   9286,   5420,   1467,  -9852,   5726,
        -4463,   9882,   9855,   6154,  18809,  -7570,  -3482,  -7332,
        -3115,   1022,   1986,   7810,  -4242,   7409,  -4718,  -3562,
         -434,  12746,   1909,   5704,  -2933,  14854,  -7138,  10993,
        17998,   4042,   -832,   7868,  -1003,  14596,  -7154,   4786,
        -3853,  -1195,  11525,   5062,  -8585,  -4646,  -2671,   4514,
         7467,   5717,  -4945,  -5808,  -1625,   1618,   6632,   7261,
         -150,   2363,   5000,   3867,  11388,   3069,   7115, -18664,
       -11462,  -1929,   2188,   3601,   4917,  -1916,   2104,  -2801,
        -4024,  -5225,   1019,   2320,  -7145,   6620,  -7998,  -4771,
        -6593,   5967,  -1835,  -1638,  -8879,  -5924,    219,    698,
         5535,    207, -18536,  -7176,  17890,  -4906,  -2600, -11301,
        20243,   2367,    587,  12012,    643,  -7090,   -598,  -2553,
        12345,  -7295,  -4181,  -6335,  -9463,  -5867,   2439,   2532,
      

In [5]:
y

array([-16508,   2478,   3515,   5364,  -4384,  -6482, -10675,   5372,
         6219,   9170,   6066,  12958,   5320,   4692,   2490, -13918,
          427,  -2340,  -2719,  -3679,  -1055, -15020,  -4362, -14098,
       -16823,   9443,  -9983,  -2793, -12924,   9181,  14357,   8604,
         2475,   1941,  -1032, -10003,  -6795,  -4177,   1531, -21650,
         3147, -11965,   2702,   3689,  -1963,  -6735, -16125,   -680,
        10188,  -4535, -23408,   8503,   4969,  -2346,   1839,  -5496,
         4698,  -8531,    759,  10408,   2790,   9064,  -3052,  -7809,
          -60,   8928,   3731,   7731,   4980,  11867,  -6397,  -1221,
        10542,   8245,   2672,  13680,   6439,   7669,  17691, -18345,
       -21586,    262,  -1048,  10555,  -3763,  -5072,   5933,   5811,
        -5660, -13243, -11868,   1003,   5175,  11692,  11571,    324,
        10985,   5753,    919,   8974, -10931,   8265,   3987,   3548,
        -6452,    169, -10740,  -1539,  15030, -13081,  -3254,   -972,
      

In [6]:
# Now create the shares

# random nonces, values in the same range as the embeddings after scaling
r = rng.integers(-30000, 30000, 128)
r

array([ 15171, -20268,  -1642,  -7067,  12123,   2691,  24042,   6836,
        23327,  29941,  18876,  16442,  15019, -13634,  -9084,   2347,
       -11805, -11072,  14033,  25948,  17573,  16380, -28445,  22127,
        17116,  29213,   1837, -14380, -28687,  16323,  -2549,  10011,
       -18245, -26726,  11995,   8765,  -7395,  -3899,  -5189,  -5920,
       -29890, -10534,  26611, -19613,   -101,   8116, -13512,   9852,
        -1689,  19731,  21610, -15688, -13517,   5467, -12183, -10634,
         4456,   6386,  12281,   2268,  26098,  20925, -21550, -19598,
        25181,  20171,  26360, -27762,   3311,  -6774, -16094,  -5613,
        11455, -24897,  -1644,  10886,  24448,  -1045,  14236,   7961,
        -3222,  18980,  -6023,  29903, -26093,  -3718,  -2442,  12428,
         4879, -10710,  13391,   1472,  -2457,  21100,  -8597,  23604,
        22967,  15754,   1410, -20259,  15099,  24542,  22472,  -2850,
       -13104,  20732,  -9680,  15732, -20734,  23182,  18968,   4588,
      

In [7]:
# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

y_0

array([-31545, -18054,  -3027,  -3951, -15941,  -5075, -29785,   3656,
        17236,  22311,  24078,  29348,  11875, -10006, -10946, -16247,
       -12216,   8732, -15440, -27395, -16572,  -1368,  32277, -24959,
         -875,  22270,  -8660,  12995,  17013,   7198, -12770,   1671,
       -20208, -28657, -10973,  -1328,   1640,   8042,  -4544,  17294,
       -30859,   1945,  28029, -17142,   1998,  -1531,   2619,  -9436,
        -8533, -23718,  -3846,  -7281, -10150,  -7283, -10426,  15614,
          818, -14753,  11534,   8308,  28436,  29397,  24518,  21005,
       -25191,  27691,  26731, -29251,   8091, -13359,   9761,   4392,
         1425, -16758,  -3100,   8182,  18087,  -6626,  29319, -22706,
        22724,  19234,   5009,  24052,  27486,   7498,  -7845,   9791,
        -1301,   6767,  -6677,   1579,  -7600,  32704,  -3240,  23920,
        29534,  11251,   1557, -27693,  -4170,  32663,  22619,  -1790,
        10780,  20565,   3132, -15223, -27212, -27031, -18094,  -4648,
      

In [8]:
# To check that the xoring works, we can XOR y_0 with y_1 and we expect to obtain y

y == np.bitwise_xor(y_0, y_1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [9]:
# Let's run the ABY code (I am providing y for verification, it's not needed nor used in the circuit)

output = run_sfe(x, y, y_0, y_1)


In [10]:
"""inspect the results
After the statistics from ABY I am printing:
- the input x,y and the share (in this case output is from the server,so we see share y_1)
- the verification results and the circuit results
"""
# Unfortunately, circuit result is incorrect.
print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
s_product nvals: 128
s_product bitlen: 64
Online time is distributed as follows: 
Bool: local gates: 270.3640000000, interactive gates: 312.9220000000, layer finish: 206.2940000000
Yao: local gates: 5.5410000000, interactive gates: 4.6240000000, layer finish: 2.3260000000
Yao Rev: local gates: 4.8740000000, interactive gates: 4.4850000000, layer finish: 2.0570000000
Arith: local gates: 5.6370000000, interactive gates: 4.6430000000, layer finish: 5.1130000000
SPLUT: local gates: 4.9980000000, interactive gates: 4.6110000000, layer finish: 26.7410000000
Communication: 1588.8240000000

Complexities: 
Boolean Sharing: ANDs: 5975939 (1-bit) ; Depth: 10990
Total Vec AND: 5975939
Total Non-Vec AND: 5975939
XOR vals: 5896327 gates: 1386811
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 24576, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing: OT-gates:

In [11]:
# Since we are getting 'inf' values, I scale by a smaller amount and see if that helps
# Let's go extreme and round the floats

x = a.round().astype(int)
y = b.round().astype(int)


x

array([-1,  1,  1,  1,  1,  0, -1,  1,  0,  1,  1,  1,  2, -1,  0, -1,  0,
        0,  0,  1,  0,  1,  0,  0,  0,  1,  0,  1,  0,  1, -1,  1,  2,  0,
        0,  1,  0,  1, -1,  0,  0,  0,  1,  1, -1,  0,  0,  0,  1,  1,  0,
       -1,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  1, -2, -1,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0, -1,  1, -1,  0, -1,  1,  0,  0, -1,
       -1,  0,  0,  1,  0, -2, -1,  2,  0,  0, -1,  2,  0,  0,  1,  0, -1,
        0,  0,  1, -1,  0, -1, -1, -1,  0,  0,  0, -1,  2,  0,  1,  0,  0,
        0,  1,  0,  0,  0,  1,  0,  0,  1])

In [12]:
# Now create the shares, everything the same way as above

# random nonces, values in the same range as the embeddings after scaling
r = rng.integers(-3, 3, 128)

# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

# To check that the xoring works, we can XOR y_0 with y_1 and we expect to obtain y
y == np.bitwise_xor(y_0, y_1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [13]:
# Let's run the ABY code (I am providing y for verification, it's not needed nor used in the circuit)

output = run_sfe(x, y, y_0, y_1)


In [14]:
# inspect the results

# Unfortunately, circuit result is incorrect.
print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
s_product nvals: 128
s_product bitlen: 64
Online time is distributed as follows: 
Bool: local gates: 205.4120000000, interactive gates: 227.6930000000, layer finish: 147.3740000000
Yao: local gates: 4.1610000000, interactive gates: 3.3780000000, layer finish: 1.6280000000
Yao Rev: local gates: 3.5980000000, interactive gates: 3.3880000000, layer finish: 1.5550000000
Arith: local gates: 4.2370000000, interactive gates: 3.5320000000, layer finish: 3.5650000000
SPLUT: local gates: 3.7480000000, interactive gates: 3.4630000000, layer finish: 20.2480000000
Communication: 1032.8790000000

Complexities: 
Boolean Sharing: ANDs: 5975939 (1-bit) ; Depth: 10990
Total Vec AND: 5975939
Total Non-Vec AND: 5975939
XOR vals: 5896327 gates: 1386811
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 24576, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing: OT-gates:

In [15]:
# I discovered by accident that if the vectors are only 0s and 1s then the circuit works as expected...
# see example

# some arrays with only 0s or 1s
x = rng.integers(0, 2, 128)
y = rng.integers(0, 2, 128)
r = rng.integers(0, 2, 128)

# create the shares in the same way as before

# y_1 is the server's share, simply the nonces
y_1 = r

# y_0 is the mb's share, it's the nonces XORed with y
y_0 = np.bitwise_xor(y, r)

output = run_sfe(x, y, y_0, y_1)

print(output.stdout)

INPUT FILE NAME: input_vecs.txt
OUTPUT FILE NAME: 
s_product nvals: 128
s_product bitlen: 64
Online time is distributed as follows: 
Bool: local gates: 162.2930000000, interactive gates: 179.0490000000, layer finish: 110.6040000000
Yao: local gates: 3.4560000000, interactive gates: 3.1720000000, layer finish: 1.3480000000
Yao Rev: local gates: 2.9190000000, interactive gates: 2.8540000000, layer finish: 1.3140000000
Arith: local gates: 3.3060000000, interactive gates: 2.8830000000, layer finish: 2.5250000000
SPLUT: local gates: 3.0230000000, interactive gates: 2.9300000000, layer finish: 16.3380000000
Communication: 744.6760000000

Complexities: 
Boolean Sharing: ANDs: 5975939 (1-bit) ; Depth: 10990
Total Vec AND: 5975939
Total Non-Vec AND: 5975939
XOR vals: 5896327 gates: 1386811
Comb gates: 0, CombStruct gates: 0, Perm gates: 0, Subset gates: 24576, Split gates: 0
Yao: ANDs: 0 ; Depth: 0
Reverse Yao: ANDs: 0 ; Depth: 0
Arithmetic Sharing: MULs: 0 ; Depth: 0
SP-LUT Sharing: OT-gates: 